# How to use the invenio API

- run API and UI with:

  ```bash
  ./run.sh
  ```

## Useful links

- [Invenio API requests](https://invenio.readthedocs.io/en/latest/getting-started/quickstart/crud-operations.html)
- [Invenio REST API](https://inveniordm.docs.cern.ch/reference/rest_api_index/) (they are lying about authentication only with Access Tokens, you can authenticate as shown below)
- [NRP Docs](https://nrp-cz.github.io/docs)
- [NRP models](https://narodni-repozitar.github.io/developer-docs/docs/technology/invenio/nrp-toolchain/edit-metadata)

## Actions

In [ ]:
import requests
import json

API_ENDPOINT = 'https://mdrepo.eu/api'
# API_ENDPOINT = 'https://inveniordm.web.cern.ch/api'

# test if API is up
response = requests.get(f'{API_ENDPOINT}/login')
if response.status_code != 405:
    raise Exception('API returned with code', response.status_code)

### Login

In [ ]:
data = {
    'email': 'test@test.com',
    'password': '123456'
}

response = requests.post(
    f'{API_ENDPOINT}/login',
    data=data
)
print(response.json())

session_cookie = response.cookies['session']
print(response.cookies)
print('Session cookie:', session_cookie)

### Search

In [ ]:
params = {
    'q': 'LRIPCCPVNLKRLLVVVVVVVLVVVVIVGALLMGL',
    'size': 10,
    'page': 1
}
response = requests.get(f'{API_ENDPOINT}/experiments', params=params)
print(response.json())

### Create record

In [ ]:
# XXX: session_cookie is required to create a record
cookies = {'session': session_cookie}

with open('demo-data/example_metadata.json', 'r') as f:
    metadata = json.load(f)

data = {
    'files': {'enabled': True},
    'parent' : {
        'communities': {'default': 'ceitec'}
    },
    'metadata': metadata,
}

response = requests.post(
    f'{API_ENDPOINT}/experiments',
    cookies=cookies,
    json=data
)

print('Status:', response.status_code)
print('Full response:', response.text)

if 'errors' in response.json():
    print('Errors:', response.json()['errors'])

if response.json().get("message", "") == "The persistent identifier does not exist.":
    print(f"⚠️Check if the group '{data['parent']['communities']['default']}' exists")

record_id = response.json().get('id', None)
print('Record ID:', record_id)

### Edit record

In [ ]:
cookies = {'session': session_cookie}

# random change to the metadata
metadata['simulations'][0]['file_identification']['description'] = "I just edited this record :P"

data = {
    'metadata': metadata,
}

response = requests.put(
    f'{API_ENDPOINT}/experiments/{record_id}/draft',
    cookies=cookies,
    json=data
)

print('Status:', response.status_code)
print('Full response:', response.text)

if 'errors' in response.json():
    print('Errors:', response.json()['errors'])

### View record

In [ ]:
# in UI
# draft:        /experiments/<record_id>/preview
# published:    /experiments/<record_id>

cookies = {'session': session_cookie}

response = requests.get(
    f'{API_ENDPOINT}/experiments/{record_id}/draft',
    cookies=cookies
)
print(response.json())

### List files of a record

In [ ]:
cookies = {'session': session_cookie}

response = requests.get(f'{API_ENDPOINT}/experiments/{record_id}/draft/files', cookies=cookies)
print(response.json())

### Upload file to a record

In [ ]:
local_file_name = 'demo-data/SPC.tpr'
remote_file_name = 'SPC.tpr'

cookies = {'session': session_cookie}

# set file metadata
data = [{'key': remote_file_name}]
response = requests.post(
    f'{API_ENDPOINT}/experiments/{record_id}/draft/files',
    cookies=cookies,
    json=data
)
print('POST metadata:', response.text)

# upload file
with open(local_file_name, 'rb') as f:
    response = requests.put(
        f'{API_ENDPOINT}/experiments/{record_id}/draft/files/{remote_file_name}/content',
        cookies=cookies,
        data=f,
        stream=True
    )
    print('PUT file:', response.text)

# commit file
response = requests.post(
    f'{API_ENDPOINT}/experiments/{record_id}/draft/files/{remote_file_name}/commit',
    cookies=cookies
)
print('POST commit:', response.text)


### Delete file from a record

In [ ]:
remote_file_name = 'SPC.tpr'

cookies = {'session': session_cookie}

response = requests.delete(
    f'{API_ENDPOINT}/experiments/{record_id}/draft/files/{remote_file_name}',
    cookies=cookies
)
print('DELETE file:', response.status_code)

### Download file from a record

In [ ]:
# XXX: record must be published (/draft only sends S3 direct url)

cookies = {'session': session_cookie}
remote_file_name = 'SPC.tpr'

response = requests.get(
    f'{API_ENDPOINT}/experiments/{record_id}/files/{remote_file_name}/content',
    cookies=cookies
)
if response.status_code >= 400:
    print(response.text)
    response.raise_for_status()

with open(remote_file_name, 'wb') as f:
    f.write(response.content)

### Publish record

In [ ]:
cookies = {'session': session_cookie}

response = requests.post(
    f'{API_ENDPOINT}/experiments/{record_id}/draft/actions/publish',
    cookies=cookies
)
print('POST publish:', response.json())